# UCI Epileptic Seizure Recognition Data Set

In [3]:
# use logistic regression
# transform data to either 1 or 0 (0 -> no seizure)

In [114]:
import pandas as pd
import torch
import torch.nn as nn
import torchvision
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [115]:
class UCIEpilepsy(Dataset):
    def __init__(self):
        self.dataset = pd.read_csv('data/data.csv')
        self.dataset = self.dataset.replace({'y':[2, 3, 4, 5]}, 0)
        self.dataset = self.dataset.iloc[:, 1:].to_numpy().astype(np.single)
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, idx):
        dataset = torch.from_numpy(self.dataset[idx,2:178])
        label = torch.from_numpy(self.dataset[idx,178:])
        return dataset, label
    
    def get_data(self):
        return torch.from_numpy(self.dataset[:,2:178])
    
    def get_targets(self):
        return torch.from_numpy(self.dataset[:,178:])
    
dataset = UCIEpilepsy()

In [116]:
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted

In [129]:
X, y = dataset.get_data(), dataset.get_targets()

n_samples, n_features = X.shape

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

tensor([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [0.],
        [0.]])


In [181]:
model = LogisticRegression(176)
learning_rate = 0.01

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

num_epochs = 10
for epoch in range(num_epochs):
    # forward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted, y_train)
    
    # backward pass
    loss.backward()
    
    # updates 
    optimizer.step()
    
    # zero gradient
    optimizer.zero_grad()
    
    if (epoch + 1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
    
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

epoch: 10, loss = 16.3341
accuracy = 0.4526
